Parte1 - CRF to NER-> Tweets e Leis

In [1]:
!pip install sklearn_crfsuite

In [0]:
import pandas as pd
import io

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
from nltk.tokenize import LineTokenizer 

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [4]:
ls gdrive/My\ Drive

'Armadilha Inteligente'/
 bottle.jpeg
'Calculo - James Stewart - 7 Edição - Volume 1.pdf'
 Certificados/
 Classroom/
'Colab Notebooks'/
 correct_image.jpg
 dataset.ptbr_CorrefPT.csv
 Desafio_Petrobras/
'EDOs exatas (1).pdf'
'EDOs exatas.pdf'
 Giordano_Estevao_Borges_Leal_e_Valter_Alberto_Melgarejo.py.gdoc
'halliday - física 1 - vol 1- 8ª ed.pdf'
'III CIT'/
'jacobi (copy).gdoc'
'Mundo Animal.doc'
 NN_evaluating/
 Pest_Dataset/
 PLN_CRF_to_NER/
 TCC/


In [5]:
%cd ./gdrive/My\ Drive/PLN_CRF_to_NER/

/content/gdrive/My Drive/PLN_CRF_to_NER


In [0]:
#Reading the csv file
df = pd.read_csv('dataset.ptbr_leis1.txt', sep=" ", header=None)

In [12]:
#Display first 10 rows
df.head(50)

,Word,BIO
0,EMENTA,O
1,:,O
2,APELAÇÃO,O
3,CÍVEL,O
4,-,O
5,AÇÃO,O
6,DE,O
7,INDENIZAÇÃO,O
8,POR,O
9,DANOS,O


In [8]:
df.columns = ["Word", "BIO"]
df.head(5) 

,Word,BIO
0,EMENTA,O
1,:,O
2,APELAÇÃO,O
3,CÍVEL,O
4,-,O


In [19]:
vetor = ['']
ap = ['']
for x in range(0,df.shape[0]-1):
  vetor.extend(ap)
df.insert(0, "Sentence #", vetor, True)
df.head(5)

,Sentence #,Word,BIO
0,,EMENTA,O
1,,:,O
2,,APELAÇÃO,O
3,,CÍVEL,O
4,,-,O


In [0]:
for x in range(0,df.shape[0]-1):
  df['Pos'][x] = 'n'

In [20]:
df.head(5)

,Sentence #,Word,BIO
0,,EMENTA,O
1,,:,O
2,,APELAÇÃO,O
3,,CÍVEL,O
4,,-,O


In [21]:
sentenca = 1
for x in range(0,df.shape[0]-1):
  if df['Word'][x] == "......":
    sentenca = sentenca + 1
    df = df.drop([x], axis=0)
  else:
    df['Sentence #'][x] = "Sentence: " + str(sentenca)
df.head(50)

,Sentence #,Word,BIO
0,Sentence: 1,EMENTA,O
1,Sentence: 1,:,O
2,Sentence: 1,APELAÇÃO,O
3,Sentence: 1,CÍVEL,O
4,Sentence: 1,-,O
5,Sentence: 1,AÇÃO,O
6,Sentence: 1,DE,O
7,Sentence: 1,INDENIZAÇÃO,O
8,Sentence: 1,POR,O
9,Sentence: 1,DANOS,O


In [22]:
#Displaying the unique Tags
df['BIO'].unique()

array(['O', 'B-ORGANIZACAO', 'I-ORGANIZACAO', 'B-LEGISLACAO',
       'I-LEGISLACAO', 'B-JURISPRUDENCIA', 'I-JURISPRUDENCIA', 'B-PESSOA',
       'I-PESSOA', 'B-TEMPO', 'B-LOCAL', 'I-LOCAL', 'I-TEMPO'],
      dtype=object)

In [23]:
df.groupby('BIO').size().reset_index(name='counts')

,BIO,counts
0,B-JURISPRUDENCIA,1104
1,B-LEGISLACAO,1920
2,B-LOCAL,611
3,B-ORGANIZACAO,2400
4,B-PESSOA,1525
5,B-TEMPO,1334
6,I-JURISPRUDENCIA,2864
7,I-LEGISLACAO,11119
8,I-LOCAL,806
9,I-ORGANIZACAO,4271


In [24]:
#Checking null values, if any.
df.isnull().sum()

Sentence #    0
Word          0
BIO           0
dtype: int64

In [0]:
df = df.fillna(method = 'ffill')

In [0]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, b) for w, b in zip(s['Word'].values.tolist(),
                                                 s['BIO'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [27]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'......'

In [29]:
sentences[1]

'EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM GRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET PARQUET INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA .'

In [30]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('EMENTA', 'O'), (':', 'O'), ('APELAÇÃO', 'O'), ('CÍVEL', 'O'), ('-', 'O'), ('AÇÃO', 'O'), ('DE', 'O'), ('INDENIZAÇÃO', 'O'), ('POR', 'O'), ('DANOS', 'O'), ('MORAIS', 'O'), ('-', 'O'), ('PRELIMINAR', 'O'), ('-', 'O'), ('ARGUIDA', 'O'), ('PELO', 'O'), ('MINISTÉRIO', 'B-ORGANIZACAO'), ('PÚBLICO', 'I-ORGANIZACAO'), ('EM', 'O'), ('GRAU', 'O'), ('RECURSAL', 'O'), ('-', 'O'), ('NULIDADE', 'O'), ('-', 'O'), ('AUSÊNCIA', 'O'), ('DE', 'O'), ('INTERVENÇÃO', 'O'), ('DO', 'O'), ('PARQUET', 'O'), ('PARQUET', 'O'), ('INSTÂNCIA', 'O'), ('A', 'O'), ('QUO', 'O'), ('-', 'O'), ('PRESENÇA', 'O'), ('DE', 'O'), ('INCAPAZ', 'O'), ('-', 'O'), ('PREJUÍZO', 'O'), ('EXISTENTE', 'O'), ('-', 'O'), ('PRELIMINAR', 'O'), ('ACOLHIDA', 'O'), ('-', 'O'), ('NULIDADE', 'O'), ('RECONHECIDA', 'O'), ('.', 'O')]


In [31]:
sentences = getter.sentences
sentences

[[('......', 'I-JURISPRUDENCIA')],
 [('EMENTA', 'O'),
  (':', 'O'),
  ('APELAÇÃO', 'O'),
  ('CÍVEL', 'O'),
  ('-', 'O'),
  ('AÇÃO', 'O'),
  ('DE', 'O'),
  ('INDENIZAÇÃO', 'O'),
  ('POR', 'O'),
  ('DANOS', 'O'),
  ('MORAIS', 'O'),
  ('-', 'O'),
  ('PRELIMINAR', 'O'),
  ('-', 'O'),
  ('ARGUIDA', 'O'),
  ('PELO', 'O'),
  ('MINISTÉRIO', 'B-ORGANIZACAO'),
  ('PÚBLICO', 'I-ORGANIZACAO'),
  ('EM', 'O'),
  ('GRAU', 'O'),
  ('RECURSAL', 'O'),
  ('-', 'O'),
  ('NULIDADE', 'O'),
  ('-', 'O'),
  ('AUSÊNCIA', 'O'),
  ('DE', 'O'),
  ('INTERVENÇÃO', 'O'),
  ('DO', 'O'),
  ('PARQUET', 'O'),
  ('PARQUET', 'O'),
  ('INSTÂNCIA', 'O'),
  ('A', 'O'),
  ('QUO', 'O'),
  ('-', 'O'),
  ('PRESENÇA', 'O'),
  ('DE', 'O'),
  ('INCAPAZ', 'O'),
  ('-', 'O'),
  ('PREJUÍZO', 'O'),
  ('EXISTENTE', 'O'),
  ('-', 'O'),
  ('PRELIMINAR', 'O'),
  ('ACOLHIDA', 'O'),
  ('-', 'O'),
  ('NULIDADE', 'O'),
  ('RECONHECIDA', 'O'),
  ('.', 'O')],
 [('II', 'O'), ('.', 'O')],
 [('NULIDADE', 'O'), ('NÃO', 'O'), ('RECONHECIDA', 'O'), ('

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    bio = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        bio1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),

        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        bio1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),

        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for bio, label in sent]

def sent2tokens(sent):
    return [token for bio, label in sent]
  

In [0]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [37]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

In [39]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9836625428213831


In [40]:
report = flat_classification_report(y_test, y_pred)
print(report)

                  precision    recall  f1-score   support

B-JURISPRUDENCIA       0.92      0.83      0.87       310
    B-LEGISLACAO       0.93      0.89      0.91       610
         B-LOCAL       0.91      0.84      0.88       171
   B-ORGANIZACAO       0.93      0.90      0.92       716
        B-PESSOA       0.96      0.91      0.93       475
         B-TEMPO       0.94      0.90      0.92       395
I-JURISPRUDENCIA       0.92      0.86      0.89       829
    I-LEGISLACAO       0.93      0.96      0.95      3549
         I-LOCAL       0.95      0.84      0.89       235
   I-ORGANIZACAO       0.93      0.94      0.93      1269
        I-PESSOA       0.98      0.96      0.97       960
         I-TEMPO       0.98      0.92      0.95       312
               O       0.99      0.99      0.99     58978

        accuracy                           0.98     68809
       macro avg       0.94      0.90      0.92     68809
    weighted avg       0.98      0.98      0.98     68809

